# Classificação de livros utilizando o NMF

* O objetivo deste trabalho é desenvolver um programa que seja capaz de:
    
> 1. Receber varios livros como entrada.
>2. Contar quantas vezes cada palavra aparece em cada livro.
>3. Gerar uma matriz **Palavra X Livro**, que armazena a quantidade de cada palavra em cada livro.
>4. Agrupar esses livros baseado em quão similares eles são.

* O agrupamento mencionado no item 4 deverá ser realizado atravez de uma fatoração da matriz **Palavra X Livro** em duas novas matrizes, uma matriz de   **Palavras X Posto** e outra de **Posto X Livro**, o significado desse posto será discutdo mais adiante. Essa fatoração deverá ser realizada pelo NMF.

*   O NMF é um método de fatoração de matrizes, que recebe de entrada uma matriz M **não negativa** (que só tem valores positivos armazenados dentro dela) e manda como saída duas matrizes W e H, também não negativas, tais que M≈WxH.
> * O diferencial do NMF é que, como tanto a matriz M quanto as matrizes W e H só possuem elementos positivos, podemos descrever os elementos de M como uma soma dos elementos de W e H
> * Essa propriedade faz do NMF um excelente método de fatoração de matrizes no que diz respeito à interpretação dos seus resultados por um ser humano, pois podemos interpretar os conteudos de W e H como pequenas partes do conteudo de M, como veremos a seguir.



# Imports
* Pandas: biblioteca de manipulação de dados, usamos ele para criar e manipular as matrizes de  **Palavra X Livro**, **Palavra X Posto** e **Posto X Livro**
* sklearn: dessa biblioteca importamos o algoritmo do NMF que será implementado, pois como o algoritmo por si só é complexo o suficiente para render outro trabalho à parte, decidiu-se que o foco deste trabalho seria apenas na implementação do NMF e suas interpretações.

In [1]:
import pandas as pd
from sklearn.decomposition import NMF

# Alfabeto e Sujeiras
* O alfabeto é o conjunto de signos que o programa de leitura será capaz de ler, foi selecionado um alfabeto para que o programa automaticamnte ignore caracteres como pontuação ou numerais. Além disso, todos os livros lidos são em inglês, para evitar problemas com acentuação..
* A sujeira são as "stopwords", palavras que não acrescentam em nada o contexto geral do livro (palavras como artigos, preposições, pronomes, etc...). Além das stopwords, foram incluidas alguns erros de digitação que foram pegos manualmente atravez de uma inspeção superficial na matriz **Palavra X Livro**

In [2]:
alfabeto = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
sujeira = ['yi', 'yf', 'y', 'x', 'wouldnt', 'would', 'w', 'v', 'u', 'ti', 'p', 'one', 'o', 'nbody', 'must', 'maybe', 'may', 'm', 'l', 'j', 'im', 'id', 'h', 'f', 'daeneryss', 'cs', 'c', 'b', 'll', 'd', 're', '', 'aaa', "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", 'doesnt', "did", 'didnt' "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]


# Funções utilizadas

*   **removerCaracterEspeciais**
>* Recebe como entrada uma palavra e compara cada caracter dela com os do array "alfabeto" definido anteriormente
>* Todos os caracteres da palavra que não estejam no alfabeto são retirados dela
>* A palavra "limpa" é retornada como saída da função

*   **contadorDePalavras**
>* Recebe como entrada o nome do livro que será lido e abre o arquivo do livro na pagina do google drive
>* "Lê" o livro, pegando todo seu conteudo e separando em um vetor de palavras
>* "Limpa" cada palavra, atravez da função  **removerCaracterEspeciais** e conta quantas vezes cada palavra aparece
>* Retorna um dicionario, onde cada chave é uma palavra e o valor associado à essa chave é a quantidade de vezes que ela apareceu

*   **removerSujeira**
>* Recebe como entrada a matriz **Palavra X Livro** e remove todas as palavras da matriz que estejam contidas no array de "sujeira" definido anteriormente
>* Retorna a matriz **Palavra X Livro** "limpa", sem as palavras contidas no array de sujeira

In [3]:
def removerCaracterEspeciais(palavra):
    palavraNova = ""
    for letra in palavra:
        if letra in alfabeto:
            palavraNova += letra
    return palavraNova

In [4]:
def contadorDePalavras(nomeDoLivro):
    livro = open(nomeDoLivro, 'r', encoding="ISO-8859-1")
    quantidadePorPalavra = {}

    for linha in livro.readlines():
        palavrasNaLinha = linha.split()
        for palavra in palavrasNaLinha:
            palavra = palavra.lower()
            palavra = removerCaracterEspeciais(palavra)
            if palavra in quantidadePorPalavra.keys():
                quantidadePorPalavra[palavra] += 1
            else:
                quantidadePorPalavra[palavra] = 1
    livro.close()
    return quantidadePorPalavra

In [5]:
def removerSujeira(dataframe):
    for palavra in list(dataframe.index):
        if palavra in sujeira:
            dataframe.drop(palavra, axis=0, inplace=True)
    return


# Cria a matriz de Palavra X livro
* palavraPorLivro: uma matriz vazia que será preenchida com as relações de quantas vezes cada palavra aparece em cada livro
* arquivosDeLivros: array com os nomes dos arquivos dos livros que serão lidos

In [6]:

palavraPorLivro = pd.DataFrame()
arquivosDeLivros = ['Foundation', 'Found_Emp', 'Sec_Found', 'Sociedade_do_anel', 'duas_torres', 'Retorno_Rei', 'Lookin_for_Alaska', 'Paper_Town', 'tartarugas_abaixo', 'querido_joao']

# Preenche a matriz de Palavra X Livro
* Lê o conteúdo dos arquivos de livros e preenche a matriz **Palavra X Livro** com as relações de quantas vezes cada palavra aparece em cada livro

In [7]:
for nomeDoLivro in arquivosDeLivros:
    quantidadePorPalavra = contadorDePalavras('./Dados/'+nomeDoLivro+".txt")
    ocorrenciaNoLivro = pd.DataFrame({'Palavra': list(quantidadePorPalavra.keys()), nomeDoLivro: list(quantidadePorPalavra.values())})
    palavraPorLivro = palavraPorLivro.append(ocorrenciaNoLivro, sort='False')

# Remove as sujeiras da matriz Palavra X Livro

In [8]:
palavraPorLivro = palavraPorLivro.groupby('Palavra').sum()
removerSujeira(palavraPorLivro)

# Fatora a matriz de Palavra X Livro em duas novas matrizes

* model: modelo que  vai realizar a fatoração NMF na matriz de **Palavra X Livro**
>* O atributo "n_components" informa para o modelo qual é o posto desejado das matrizes **Palavra X Posto** e **Posto X Livro**
* palavraPorGenero: a matriz **Palavra X Posto**
* livroPorGenero: a matiz **Posto X Livro**


In [9]:
model = NMF(n_components=3)
palavraPorGenero = pd.DataFrame(data=model.fit_transform(palavraPorLivro)).join(pd.DataFrame({'palavras': list(palavraPorLivro.index)})).groupby('palavras').sum()
livroPorGenero = model.components_
livroPorGenero = pd.DataFrame(data=livroPorGenero, columns=['Fundacao e imperio', 'Fundacao','Quem eh voce Alaska?' ,'Cidade de Papel' ,'Retorno do Rei', 'Segunda fundacao', 'Sociedade do Anel', 'Duas Torres', 'querido joao', 'Tartarugas Embaixo'])

# Vendo o significado do posto
* Abaixo, imprimiu-se a matriz **Posto X Livro** e verificamos que certos grupos livros tem valores muito altos em certas linhas
* Graças à propriedade da não negatividade do NMF o conteúdo dos livros deve ser, aproxidamamente, uma soma dos valores de cada linha para aquele livro
* Sendo assim, podemos atribuir um significado para essas linhas e dizer que os livros possuem "muito" daquela linha em sua composição. Um exemplo seria interpretar cada linha, ou seja, o posto, como sendo um **Genero**

In [10]:
livroPorGenero

,Fundacao e imperio,Fundacao,Quem eh voce Alaska?,Cidade de Papel,Retorno do Rei,Segunda fundacao,Sociedade do Anel,Duas Torres,querido joao,Tartarugas Embaixo
0,0.659117,0.189653,0.460569,0.00000,35.045000,0.000000,48.807015,40.159293,0.000000,0.141330
1,22.129976,46.180781,0.108857,0.00000,0.940298,23.559015,0.000000,2.120044,2.929188,0.191076
2,0.000000,0.000000,27.292287,32.51002,0.000000,0.000000,1.130364,3.001595,30.737744,25.495285


# Matriz Genero X Livro

* Vamos agora atribuir para cada linha um genero diferente, os nomes de cada linha foram escolhidos baseado num conhecimento dos livros por parte do autor do trabalho

In [11]:
livroPorGenero = livroPorGenero.join(pd.DataFrame({'Genero': ['Fanatsia', 'Sci-Fi', 'Drama']}))
livroPorGenero = livroPorGenero.groupby('Genero').sum()

# Analise do resultado

* Agora, ao imprimir novamente a matriz **Genero X Livro**, mas dessa vez atribuindo nome para as linhas, podemos ver claramente que livros de generos similares ficam, de fato, agrupados na mesma linha
* Entretanto, ainda há um problema na leitura dessa matriz. Se pararmos para ver o somatório dos valores de cada coluna, perceberemos que eles variam bastante de livro para livro. Isso se deve porque o somatório de cada coluna depende do **tamanho do livro**. Isso leva a alguns questionamentos:
>1. Essa matriz é a melhor maneira de ler os resultados?
>2. Baseado nessa matriz o livro "Fundação" possui o dobro de Sci-Fi que o livro "Fundação e Imperio", mas será que isso é mesmo verdade?
>3. O tamanho dos livros deve mesmo ser levado em conta na hora de ver o quanto de cada genero ele possui?

In [12]:
livroPorGenero

,Fundacao e imperio,Fundacao,Quem eh voce Alaska?,Cidade de Papel,Retorno do Rei,Segunda fundacao,Sociedade do Anel,Duas Torres,querido joao,Tartarugas Embaixo
Genero,,,,,,,,,,
Drama,0.000000,0.000000,27.292287,32.51002,0.000000,0.000000,1.130364,3.001595,30.737744,25.495285
Fanatsia,0.659117,0.189653,0.460569,0.00000,35.045000,0.000000,48.807015,40.159293,0.000000,0.141330
Sci-Fi,22.129976,46.180781,0.108857,0.00000,0.940298,23.559015,0.000000,2.120044,2.929188,0.191076


# Normalização
* Para resolver as questões levantadas anteriormente, pensou-se na ideia de normalizar os resultados da matriz **Genero X Livro**
* Essa normalização foi feita atravez da função **normalizar**, que pega todos os valores de cara coluna e divide pelo somatório dos valores dessa coluna, dessa forma o tamanho do livro deixaria de ser um fator importante e a matriz **Genero X Livro** normalizada passa a armazenar o percentual de cada gênero contido em cada livro

In [13]:
def normalizar(coluna):
  total = 0
  for i in list(coluna):
    total += i
  for i in range(coluna.size):
    coluna[i] = coluna[i]/total
    
def test(x):
  return 2*x

# Resultados Finais

* Temos agora uma matriz **Gênero X Livro** normalizada, onde podemosver de forma clara quanto de cada gênero cada livro tem dentro dele. 
* Essa matriz mostra como o NMF é uma ferramenta poderosa quando queremos agrupar determinado conjunto de dados baseado em alguma similaridade que esses dados tenham entre si.
* Embora não seja o algoritmo mais preciso para fatoração de matrizes, o poder do NMF se encontra na facilidade que é pro serhumano interpretar seus resultados, como visto ao longo desse trabalho.

In [14]:
livroPorGeneroNormalizado = livroPorGenero.copy()
livroPorGeneroNormalizado.apply(normalizar)
livroPorGeneroNormalizado

,Fundacao e imperio,Fundacao,Quem eh voce Alaska?,Cidade de Papel,Retorno do Rei,Segunda fundacao,Sociedade do Anel,Duas Torres,querido joao,Tartarugas Embaixo
Genero,,,,,,,,,,
Drama,0.000000,0.00000,0.979562,1.0,0.00000,0.0,0.022636,0.066288,0.912995,0.987130
Fanatsia,0.028922,0.00409,0.016531,0.0,0.97387,0.0,0.977364,0.886892,0.000000,0.005472
Sci-Fi,0.971078,0.99591,0.003907,0.0,0.02613,1.0,0.000000,0.046820,0.087005,0.007398


In [15]:
palavraPorLivro

,Found_Emp,Foundation,Lookin_for_Alaska,Paper_Town,Retorno_Rei,Sec_Found,Sociedade_do_anel,duas_torres,querido_joao,tartarugas_abaixo
Palavra,,,,,,,,,,
aback,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
abandon,0.0,2.0,2.0,2.0,1.0,2.0,3.0,3.0,0.0,1.0
abandoned,0.0,2.0,2.0,21.0,1.0,2.0,2.0,0.0,1.0,0.0
abandoning,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
abandonment,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
zoologist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
zoom,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
zoomed,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
palavraPorGenero

,0,1,2
palavras,,,
aback,0.006153,0.021874,0.000000
abandon,0.055397,0.042298,0.041814
abandoned,0.016418,0.036940,0.224043
abandoning,0.000000,0.000000,0.033532
abandonment,0.000000,0.021852,0.000000
...,...,...,...
zoologist,0.000000,0.000000,0.074834
zoom,0.000000,0.021689,0.006820
zoomed,0.000000,0.000000,0.017026
